In [1]:
import cv2
import numpy as np


def get_mask(points, shape):
    """
    返回遮罩，在不规则多边形区域中为白色，外面为黑色
    :param points:
    :param shape:
    :return:
    """
    mask = np.zeros(shape, np.uint8)
    pts = points.reshape((-1, 1, 2))
    cv2.fillPoly(mask, [pts], (255, 255, 255))
    return mask


def get_roi(img, points):
    """
    返回roi区域
    :param img:
    :param points:
    :return:
    """
    mask = get_mask(points, img.shape)
    roi = cv2.bitwise_and(img, mask)
    return roi


def callback(*args, **kargs):
    print('enter callback, args is {}, kargs is{}'.format(args, kargs))

    
def difference(img1, img2, threshold, callback):
    """
    差分法，当两帧相差大于threshold阈值时调用回调函数callback
    :param img1:
    :param img2:
    :param threshold:
    :param callback:
    :return:
    """
    diff = cv2.absdiff(img1, img2)
    value = np.sum(diff)
    if value >= threshold:
        callback(value)
    return diff

In [2]:
fps = 30
exit = 27

In [9]:
# roi
cap = cv2.VideoCapture(0)
ret, image = cap.read()
shape = image.shape
points = np.array([[10, 10], [50, 500], [400, 400], [500, 10]], np.int32)
mask = get_mask(points, shape)
while ret:
    roi = cv2.bitwise_and(image, mask)
    #     roi = get_roi(image, points)
    cv2.imshow('roi', roi)
    ret, image = cap.read()
    if cv2.waitKey(fps) & 0xFF == exit:
        break
cap.release()
cv2.destroyAllWindows()

In [6]:
# 差分法
cap = cv2.VideoCapture(0)
ret, image = cap.read()
shape = image.shape
points = np.array([[10, 10], [50, 500], [400, 400], [500, 10]], np.int32)
mask = get_mask(points, shape)
roi1 = cv2.bitwise_and(image, mask)
while ret:
    ret, image = cap.read()
    roi2 = cv2.bitwise_and(image, mask)
    #     roi = get_roi(image, points)
    diff = difference(roi1, roi2, 8002850, callback)
    cv2.imshow('name', diff)
    roi1=roi2
    
    if cv2.waitKey(fps) & 0xFF == exit:
        break
cap.release()
cv2.destroyAllWindows()

enter callback, args is (10452244,), kargs is{}
enter callback, args is (8021165,), kargs is{}


In [8]:
# 高斯混合模型
cap = cv2.VideoCapture(0)
ret, image = cap.read()
shape = image.shape
points = np.array([[10, 10], [50, 500], [400, 400], [500, 10]], np.int32)
mask = get_mask(points, shape)
fgbg = cv2.createBackgroundSubtractorMOG2()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
while ret:
    ret, image = cap.read()
    roi = cv2.bitwise_and(image, mask)
    fgmask = fgbg.apply(roi)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    cv2.imshow('gaussian', fgmask)
    
    if cv2.waitKey(fps) & 0xFF == exit:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
# 光流估计，需要先做角点检测，再传入光流估计中


In [18]:
cap.release()
cv2.destroyAllWindows()